In [4]:
import pandas as pd
import os, sys
import time
from tqdm import tqdm 
import importlib
sys.path.append(os.path.abspath(".."))
from utils import images_clf


In [5]:
importlib.reload(images_clf)
from utils.images_clf import check_pic_url, download_image

In [13]:
df=pd.read_csv("../A_preprocess\data_jo_3_processed\listings_jo_paris2024_processed1.csv")
print(len(df))

74579


C:\Users\yeliu\AppData\Local\Temp\ipykernel_8248\742478674.py:1: DtypeWarning: Columns (78,80) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("../A_preprocess\data_jo_3_processed\listings_jo_paris2024_processed1.csv")


# test:

## stratifid sampling


In [ ]:
df=df.drop_duplicates(subset=['host_id'])
print(len(df))#-5K

N_total=2000
strata = (
    df
    .groupby(["is_changed", "has_host_about", "lang"])
    .size()
    .reset_index(name="N_s")
)
# 每个类别按照原df比例在smaple中应该拥有的数量 = 按比例 * 总数
strata["n_sample"] = (strata["N_s"] / strata["N_s"].sum() * N_total).round().astype(int)
print(strata,"\n")


sampled_list = []
for _, row in strata.iterrows():
    cond = (
        (df["is_changed"] == row["is_changed"]) &
        (df["has_host_about"] == row["has_host_about"]) &
        (df["lang"] == row["lang"])
    )
    subset = df[cond]

    # 如果某层数量不足，直接全取
    n = min(row["n_sample"], len(subset))

    sampled = subset.sample(n=n, random_state=42)
    sampled_list.append(sampled)

sampled_df = pd.concat(sampled_list, ignore_index=True)
print(sampled_df,"\n")

id_url_df=sampled_df.copy()[:][['host_id','host_picture_url']]
id_url_list= list(zip(id_url_df["host_id"], id_url_df["host_picture_url"]))
print(len(id_url_list))

53414
   is_changed  has_host_about         lang    N_s  n_sample
0       False               0      no_text  23019       862
1       False               1           en   5844       219
2       False               1           fr   8203       307
3       False               1      no_text     98         4
4       False               1  other_langs    423        16
5        True               0      no_text  10744       402
6        True               1           en   1896        71
7        True               1           fr   3010       113
8        True               1      no_text     23         1
9        True               1  other_langs    154         6 

      Unnamed: 0                   id  \
0          70606  1048307549455709351   
1          53957   889964538720137624   
2          55318   905706794059511080   
3           6052              6599882   
4          52030   866332686709711350   
...          ...                  ...   
1996       84812  1123977884705743498   
1997

## download:

In [ ]:
importlib.reload(images_clf)
from utils.images_clf import download_images_batch

# results = download_images_batch(id_url_list, out_dir='images_raw', max_workers=8)

success = [p for p in results.values() if p]
fail = [u for u, p in results.items() if not p]#无效url/下载失败！

print(f"[SUCCES] {len(success)} 张")
print(f"[FAIL] {len(fail)} 张：")

[WARNING] 无效 URL: https://a0.muscache.com/im/pictures/user/User-34961932/original/65ec51af-e1af-4157-bdaf-d28b92a340c6.jpeg?aki_policy=profile_x_medium


[WARNING] 无效 URL: https://a0.muscache.com/im/pictures/user/User-559574902/original/199d8914-fb04-4292-b9b8-9a21fe021ed2.jpeg?aki_policy=profile_x_medium


[WARNING] 无效 URL: https://a0.muscache.com/im/pictures/user/149760ba-c918-4ce3-8670-421de17f841e.jpg?aki_policy=profile_x_medium


[WARNING] 无效 URL: https://a0.muscache.com/im/pictures/user/331fcb46-9021-406a-9d37-8e396cdba838.jpg?aki_policy=profile_x_medium


[WARNING] 无效 URL: https://a0.muscache.com/im/users/26081843/profile_pic/1440313903/original.jpg?aki_policy=profile_x_medium


...

[WARNING] 无效 URL: https://a0.muscache.com/im/pictures/user/User-574630696/original/70bb3589-1ba1-4ff3-be3e-248af546fbaf.jpeg?aki_policy=profile_x_medium


[WARNING] 无效 URL: https://a0.muscache.com/im/pictures/user/83de5532-a161-42e9-852a-a04c754d0e30.jpg?aki_policy=profile_x_medium


[WARNING] 无效 URL: https://a0.muscache.com/im/pictures/user/a0c4fbcf-676c-477f-8196-436862275a21.jpg?aki_policy=profile_x_medium


[INFO] host 438828 pic 已存在，跳过： images_raw\438828.jpg



 [SUCCES] downloaded 2001 : 673.04 sec!

[SUCCES] 1987 张
[FAIL] 14 张：


In [8]:
results

{438828: 'images_raw\\438828.jpg',
 474546: 'images_raw\\474546.jpg',
 429406: 'images_raw\\429406.jpg',
 464019: 'images_raw\\464019.jpg',
 228508: 'images_raw\\228508.jpg',
 48733: 'images_raw\\48733.jpg',
 108444: 'images_raw\\108444.jpg',
 81840: 'images_raw\\81840.jpg',
 532468: 'images_raw\\532468.jpg',
 107514: 'images_raw\\107514.jpg'}

## split:

In [52]:
sampled_df[["is_changed","has_host_about","lang"]].value_counts(dropna=False)

is_changed  has_host_about  lang       
False       0               no_text        862
True        0               no_text        402
False       1               fr             307
                            en             219
True        1               fr             113
                            en              71
False       1               other_langs     16
True        1               other_langs      6
False       1               no_text          4
True        1               no_text          1
Name: count, dtype: int64

In [7]:
# strat = sampled_df[["is_changed","has_host_about","lang"]].apply(lambda x: "_".join([str(v) for v in x]), axis=1)
#restart!
strat.value_counts()

False_0_no_text        862
True_0_no_text         402
False_1_fr             307
False_1_en             219
True_1_fr              113
True_1_en               71
False_1_other_langs     16
True_1_other_langs       6
False_1_no_text          4
True_1_no_text           1
Name: count, dtype: int64

In [ ]:
# from sklearn.model_selection import train_test_split

# test_ratio = 0.15
# train_ratio = 0.35
# pool_ratio = 0.50   # 不用直接用，最后自动得到

# # Step 1：POOL vs REST
# pool_df, rest_df = train_test_split(
#     sampled_df,
#     test_size=pool_ratio,
#     # stratify=strat,
#     random_state=42
# )

# train_ratio_updated = train_ratio / (1 - pool_ratio) 
# train_df, test_df = train_test_split(
#     rest_df,
#     train_size=train_ratio_updated,
#     # stratify=strat,
#     random_state=42
# )
# print(f"TEST vs TRAIN vs POOL :{len(test_df)}, {len(train_df)}, {len(pool_df)}")


TEST vs TRAIN vs POOL :301, 700, 1000


In [20]:
os.path.filename('10049969.jpg')


AttributeError: module 'ntpath' has no attribute 'filename'

In [ ]:
## 在每一个images_*中生成一个csv，包括images_filenames:host_id.jpg, host_picture_url 用于将该folder中图片通过url导入labelstudio！
import numpy as np

filenames=os.listdir('images_TEST')
for file in filenames :
    fname=os.basename(file)
    df=pd.DataFrame({'filename':file,
                    "picture_url":np.where(df['host_id']) })

['10049969.jpg',
 '102571900.jpg',
 '1030199.jpg',
 '1037777.jpg',
 '1038778.jpg',
 '106294215.jpg',
 '106365215.jpg',
 '10810717.jpg',
 '10939195.jpg',
 '11392670.jpg',
 '11396528.jpg',
 '114047773.jpg',
 '116979102.jpg',
 '121006168.jpg',
 '1219743.jpg',
 '123638123.jpg',
 '124207254.jpg',
 '126146155.jpg',
 '127632443.jpg',
 '12959928.jpg',
 '13086789.jpg',
 '1353874.jpg',
 '136561923.jpg',
 '137154154.jpg',
 '13740837.jpg',
 '137535422.jpg',
 '14043547.jpg',
 '14088988.jpg',
 '14266056.jpg',
 '14627960.jpg',
 '147313991.jpg',
 '1500611.jpg',
 '150718062.jpg',
 '15083505.jpg',
 '150990981.jpg',
 '151437721.jpg',
 '15213500.jpg',
 '156918105.jpg',
 '15886494.jpg',
 '1617094.jpg',
 '162755108.jpg',
 '163126810.jpg',
 '163132965.jpg',
 '164791211.jpg',
 '16509032.jpg',
 '16767965.jpg',
 '168770756.jpg',
 '170358950.jpg',
 '17244015.jpg',
 '1725793.jpg',
 '17405620.jpg',
 '17446647.jpg',
 '17582103.jpg',
 '17589377.jpg',
 '176807469.jpg',
 '176943238.jpg',
 '17700238.jpg',
 '17715461.jp

In [30]:
import os
import json
import pandas as pd
import time
from tqdm import tqdm

start_time=time.time()
# 读取原始 host 信息
# df

# 读取 images_TEST 文件名
folder = "images_TEST"
filenames = sorted(os.listdir(folder))

records = []
for file in tqdm(filenames, desc="mapping url of images"):
    # 跳过非图片文件
    if not file.lower().endswith((".jpg", ".jpeg", ".png")):
        continue
    # 文件名：41925864.jpg → 41925864
    host_id = os.path.splitext(file)[0]

    # host_id 通常是整数，所以这里尝试转成 int
    try:
        host_id_int = int(host_id)
    except:
        print(f"[WARN] Cannot parse host_id from file: {file}")
        continue

    # 查找该 host_id 对应的行
    row = df[df["host_id"] == host_id_int]

    if row.empty:
        print(f"[WARN] host_id {host_id_int} not found in DataFrame!")
        continue

    row = row.iloc[0]

    # 构造 Label Studio task 数据
    record = {
        # "folder": folder,                # 保留原始文件名作为额外信息
        "host_id": int(row["host_id"]),
        "filename":file,
        "image": row["host_picture_url"],      # 自动加载远程 URL
    }

    records.append(record)

# 保存成 JSON
output_path = os.path.join(folder, "000filename_url_records.json")
with open(output_path, "w") as f:
    json.dump(records, f, indent=2)
end_time=time.time()
print(f"[SUCCESS] JSON filename:url saved to {output_path}, total {len(records)} items: {end_time-start_time:.2f} sec!")


mapping url of images: 100%|██████████| 300/300 [00:00<00:00, 4218.89it/s]

[SUCCESS] JSON filename:url saved to images_TEST\000filename_url_records.json, total 299 items: 0.08 sec!


In [31]:
print(len(records))
print(records[0])

299
{'host_id': 10049969, 'filename': '10049969.jpg', 'image': 'https://a0.muscache.com/im/users/10049969/profile_pic/1384614593/original.jpg?aki_policy=profile_x_medium'}


## embeddings：

In [ ]:
from PIL import Image
import os,sys
sys.path.append(os.path.abspath(".."))
import pandas as pd
from tqdm import tqdm
import importlib
from utils import images_clf


#-----------------------------------------
import torch
from transformers import CLIPProcessor, CLIPModel
# 选择设备
device = "cuda" if torch.cuda.is_available() else "cpu"

# 加载 CLIP 模型
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


In [ ]:
importlib.reload(images_clf)
from utils.images_clf import embed_folder, embed_images_save_mapping


In [ ]:
embed_images_save_mapping(model, processor, device, images_folders=["images_SAMPLE"], embeddings_folder='embeddings_test')


Embedding 'images_SAMPLE' => 'embeddings_test\emb_SAMPLE.npz': 100%|██████████| 10/10 [00:00<?, ?it/s]

[INFO] 1030199.jpg already embedded! SKIPPING!
[INFO] 1037777.jpg already embedded! SKIPPING!
[INFO] 1038778.jpg already embedded! SKIPPING!
[INFO] 1219743.jpg already embedded! SKIPPING!
[INFO] 1353874.jpg already embedded! SKIPPING!
[INFO] 1500611.jpg already embedded! SKIPPING!
[INFO] 1617094.jpg already embedded! SKIPPING!
[INFO] 1725793.jpg already embedded! SKIPPING!
[INFO] 724209.jpg already embedded! SKIPPING!
[INFO] 873444.jpg already embedded! SKIPPING!
[SUCCES] All embeddings saved and mapped: 0.05 sec!


## import to labelstudio

In [2]:
import json

In [3]:
with open("annotations_TEST/annotations_TEST_100-project-1-at-2025-11-20-19-00-41fdcddd.json") as f:
    data=json.load(f)
data[0]

{'id': 1,
 'annotations': [{'id': 1,
   'completed_by': 1,
   'result': [{'value': {'choices': ['pro']},
     'id': 'AUyNi4u7m1',
     'from_name': 'type',
     'to_name': 'image',
     'type': 'choices',
     'origin': 'manual'},
    {'value': {'choices': ['low']},
     'id': 'N_IlrhBFld',
     'from_name': 'quality',
     'to_name': 'image',
     'type': 'choices',
     'origin': 'manual'},
    {'value': {'choices': ['1']},
     'id': '8EwtDU7Vyc',
     'from_name': 'is_smiling',
     'to_name': 'image',
     'type': 'choices',
     'origin': 'manual'},
    {'value': {'choices': ['F']},
     'id': 'glAbREH2hq',
     'from_name': 'sex',
     'to_name': 'image',
     'type': 'choices',
     'origin': 'manual'},
    {'value': {'choices': ['young']},
     'id': 'WioQg1eMTs',
     'from_name': 'age',
     'to_name': 'image',
     'type': 'choices',
     'origin': 'manual'}],
   'was_cancelled': False,
   'ground_truth': False,
   'created_at': '2025-11-20T17:27:11.710177Z',
   'updated_at

# application :

In [7]:
import pandas as pd
import os, sys
import time
from tqdm import tqdm 
import importlib
sys.path.append(os.path.abspath(".."))
from utils import images_clf

## sampling + download

In [8]:
importlib.reload(images_clf)
from utils.images_clf import stratified_sampling, download_images_batch

In [9]:
df=pd.read_csv("../A_preprocess\data_jo_3_processed\listings_jo_paris2024_processed1.csv")
sampled_df, id_url_list=stratified_sampling(df, groupby=["is_changed", "has_host_about", "lang"], N_total=2000)

C:\Users\yeliu\AppData\Local\Temp\ipykernel_8248\4163720679.py:1: DtypeWarning: Columns (78,80) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("../A_preprocess\data_jo_3_processed\listings_jo_paris2024_processed1.csv")


*******************STRATIFIED SAMPLING***************************
METHODS:
regroupe df by is_changed,has_host_about,lang;
sampling proportionally 2000 pics;
=> get a list of set (host_id, host_picture_id)

===============INPUT INFO=================
len before deduplication by host_id:74579
after:53414

strata :
    is_changed  has_host_about         lang    N_s  n_sample
0       False               0      no_text  23019       862
1       False               1           en   5844       219
2       False               1           fr   8203       307
3       False               1      no_text     98         4
4       False               1  other_langs    423        16
5        True               0      no_text  10744       402
6        True               1           en   1896        71
7        True               1           fr   3010       113
8        True               1      no_text     23         1
9        True               1  other_langs    154         6

len SAMPLE/'id_url_list':

,host_id,host_picture_url,is_changed,has_host_about,lang
0,41925864,https://a0.muscache.com/im/users/41925864/prof...,False,0,no_text
1,6853322,https://a0.muscache.com/im/pictures/user/039a5...,False,0,no_text
2,410087584,https://a0.muscache.com/im/pictures/user/ad305...,False,0,no_text
3,3394353,https://a0.muscache.com/im/users/3394353/profi...,False,0,no_text
4,143767086,https://a0.muscache.com/im/pictures/user/User-...,False,0,no_text


## split + copy

In [ ]:

importlib.reload(images_clf)
from utils.images_clf import split_copy
split_copy (sampled_df, RAW_DIR = "images_raw", TEST_DIR = "images_TEST", TRAIN_DIR = "images_TRAIN", POOL_DIR = "images_POOL")

TEST vs TRAIN vs POOL :301, 700, 1000
[INFO] host 54394328 pic already copied: images_TEST\54394328.jpg
[INFO] host 273517394 pic already copied: images_TEST\273517394.jpg
[INFO] host 29254774 pic already copied: images_TEST\29254774.jpg
[INFO] host 22152688 pic already copied: images_TEST\22152688.jpg
[INFO] host 147313991 pic already copied: images_TEST\147313991.jpg
[INFO] host 2379345 pic already copied: images_TEST\2379345.jpg
[INFO] host 67432569 pic already copied: images_TEST\67432569.jpg
[INFO] host 245148121 pic already copied: images_TEST\245148121.jpg
[INFO] host 11396528 pic already copied: images_TEST\11396528.jpg
[INFO] host 498682569 pic already copied: images_TEST\498682569.jpg
[INFO] host 116979102 pic already copied: images_TEST\116979102.jpg
[INFO] host 336591839 pic already copied: images_TEST\336591839.jpg
[INFO] host 64882116 pic already copied: images_TEST\64882116.jpg
[INFO] host 2483261 pic already copied: images_TEST\2483261.jpg
[INFO] host 274790393 pic alre

## embedding + mapping

In [ ]:
#-----------------------------------------#
import torch
from transformers import CLIPProcessor, CLIPModel
# 选择设备
device = "cuda" if torch.cuda.is_available() else "cpu"

# 加载 CLIP 模型
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


#-----------------------------------------#
import importlib
from utils import images_clf  
importlib.reload(images_clf)
from utils.images_clf import embed_images_save_mapping

In [ ]:
embed_images_save_mapping(model, processor, device, 
                          images_folders=["images_TEST","images_TRAIN","images_POOL"], embeddings_folder='embeddings')


Embedding 'images_TEST' => 'embeddings\emb_TEST.npz': 100%|██████████| 299/299 [00:29<00:00, 10.11it/s]
Embedding 'images_TRAIN' => 'embeddings\emb_TRAIN.npz': 100%|██████████| 697/697 [01:10<00:00,  9.84it/s]
Embedding 'images_POOL' => 'embeddings\emb_POOL.npz': 100%|██████████| 991/991 [01:36<00:00, 10.31it/s]

[SUCCES] All embeddings saved and mapped: 196.67 sec!
